In [2]:
import numpy as np
import pandas as pd
from scipy.stats import f

# Data Input: nilai respons kerusakan
data_matrix = np.array([
    [3, 1, -2, 0],    # Day 1: A, B, C, D
    [0, 0, -1, 7],    # Day 2: B, C, D, E
    [-1, 0, 5, 3],    # Day 3: C, D, E, A
    [-1, 6, 4, 0],    # Day 4: D, E, A, B
    [5, 2, 1, -1]     # Day 5: E, A, B, C
])

treatments = [
    ['A', 'B', 'C', 'D'],
    ['B', 'C', 'D', 'E'],
    ['C', 'D', 'E', 'A'],
    ['D', 'E', 'A', 'B'],
    ['E', 'A', 'B', 'C']
]

In [4]:
#Statistik uji diketahui
a = b = 5   # number of treatments = number of blocks
k = 4       # positions per block
r = k
lambda_val = r * (k - 1) / (a - 1)
print("lambda = ", lambda_val) 
print("a = ", a)
print("b = ", b)
print("k = ", k)
print("r = ", r)

lambda =  3.0
a =  5
b =  5
k =  4
r =  4


In [5]:
# Flatten data and labels
observations = []
for i in range(5):         # block/day
    for j in range(4):     # station
        observations.append({
            "day": i + 1,
            "station": j + 1,
            "treatment": treatments[i][j],
            "value": data_matrix[i][j]
        })

df = pd.DataFrame(observations)
df

,day,station,treatment,value
0,1,1,A,3
1,1,2,B,1
2,1,3,C,-2
3,1,4,D,0
4,2,1,B,0
5,2,2,C,0
6,2,3,D,-1
7,2,4,E,7
8,3,1,C,-1
9,3,2,D,0


In [7]:
# Total N
N = len(df)
Y_total = df['value'].sum()
Y_squared_total = Y_total ** 2 / N
df['value_squared'] = df['value'] ** 2
JKT = df['value_squared'].sum() - Y_squared_total
print("JKT = ", JKT)

JKT =  134.95


In [17]:
# Posisi (Stasiun)
position_sums = df.groupby("station")['value'].sum()
JKPos = sum(position_sums ** 2) / b - Y_squared_total

# Kehadiran perlakuan per blok
presence = pd.DataFrame(0, index=range(1, 6), columns=['A', 'B', 'C', 'D', 'E'])
for i, row in df.iterrows():
    presence.loc[row['day'], row['treatment']] = 1

# Hitung phi
block_sums = df.groupby("day")['value'].sum()
treatment_sums = df.groupby("treatment")['value'].sum()
phi = {}
for treat in treatment_sums.index:
    y_j = treatment_sums[treat]
    comp = sum(presence[treat][i] * (block_sums[i] / k) for i in block_sums.index)
    phi[treat] = y_j - comp
phi_series = pd.Series(phi)

# JKP (Perlakuan diperbaiki)
JKP = (k * sum(phi_series ** 2)) / (lambda_val * a)
print("JKP = ", round(JKP, 2))


JKP =  120.37


In [18]:
# Block (Day) totals
block_sums = df.groupby("day")['value'].sum()
JKB = sum(block_sums ** 2) / k - Y_squared_total
print("JKB = ", JKB)

JKB =  6.700000000000003


In [10]:
# Position totals (Station)
position_sums = df.groupby("station")['value'].sum()
JKPos = sum(position_sums ** 2) / b - Y_squared_total
print("JKPos = ", JKPos)

JKPos =  1.3500000000000014


In [13]:
# Error (Residual)
JKS = JKT - JKP - JKB - JKPos
print("JKS = ", JKS)

JKS =  6.533333333333324


In [15]:
# Degrees of freedom
df_total = N - 1
df_treat = a - 1
df_block = b - 1
df_pos = k - 1
df_error = N - a - b - k + 2
print("df_total = ", df_total)
print("df_treat = ", df_treat)
print("df_block = ", df_block)
print("df_pos = ", df_pos)
print("df_error = ", df_error)

df_total =  19
df_treat =  4
df_block =  4
df_pos =  3
df_error =  8


In [16]:
# Mean Squares
RKP = JKP / df_treat
RKB = JKB / df_block
RKPos = JKPos / df_pos
RKS = JKS / df_error
print("RKP = ", RKP)
print("RKB = ", RKB)
print("RKPos = ", RKPos)
print("RKS = ", RKS)

RKP =  30.091666666666665
RKB =  1.6750000000000007
RKPos =  0.45000000000000046
RKS =  0.8166666666666655


In [20]:
# F-statistic
F_hit = RKP / RKS
F_table = f.ppf(1 - 0.1, df_treat, df_error)
print("F_hit = ", F_hit)

F_hit =  36.84693877551025


In [21]:
# === OUTPUT ANOVA ===
print("===== ANOVA RBSY =====")
print(f"JKT = {JKT:.2f}")
print(f"JK Perlakuan = {JKP:.2f}")
print(f"JK Blok = {JKB:.2f}")
print(f"JK Posisi = {JKPos:.2f}")
print(f"JK Sisa = {JKS:.2f}")
print(f"F hitung = {F_hit:.2f}, F tabel (0.1) = {F_table:.2f}")

===== ANOVA RBSY =====
JKT = 134.95
JK Perlakuan = 120.37
JK Blok = 6.70
JK Posisi = 1.35
JK Sisa = 6.53
F hitung = 36.85, F tabel (0.1) = 2.81


In [22]:
# === UJI DUNCAN ===
tau_hat = (k * phi_series) / (lambda_val * a)
tau_hat = tau_hat.sort_values()
tau_names = tau_hat.index.tolist()
tau_values = tau_hat.values

S = (k * RKS / (lambda_val * a)) ** 0.5
r_values = {2: 4.74, 3: 5.00, 4: 5.14, 5: 5.23}

print("\n===== UJI DUNCAN =====")
for i in range(len(tau_values)):
    for j in range(i + 1, len(tau_values)):
        diff = tau_values[j] - tau_values[i]
        p = j - i + 1
        Rp = r_values[p] * S
        status = "BERBEDA" if diff > Rp else "SAMA"
        print(f"{tau_names[j]} - {tau_names[i]} = {diff:.2f} vs Rp({p}) = {Rp:.2f} → {status}")



===== UJI DUNCAN =====
D - C = 0.40 vs Rp(2) = 2.21 → SAMA
B - C = 1.47 vs Rp(3) = 2.33 → SAMA
A - C = 4.07 vs Rp(4) = 2.40 → BERBEDA
E - C = 6.73 vs Rp(5) = 2.44 → BERBEDA
B - D = 1.07 vs Rp(2) = 2.21 → SAMA
A - D = 3.67 vs Rp(3) = 2.33 → BERBEDA
E - D = 6.33 vs Rp(4) = 2.40 → BERBEDA
A - B = 2.60 vs Rp(2) = 2.21 → BERBEDA
E - B = 5.27 vs Rp(3) = 2.33 → BERBEDA
E - A = 2.67 vs Rp(2) = 2.21 → BERBEDA
